In [18]:
import sys , os

# -----------------------------------------------------------------------------
# Project Path Setup
#    - Add project root to Python path for consistent imports
# -----------------------------------------------------------------------------



sys.path.append("..")




from model.MTCformerV3 import MTCFormer
import torch
import glob
import numpy as np
import pandas as pd
from torch.optim import Adam
from utils.augmentation import augment_data
from utils.training import  train_model , predict_optimized
from utils.CustomDataset import EEGDataset
from torch.utils.data import DataLoader
from utils.loader import Loader
from utils.preprocessing import SignalPreprocessor

print("extracting data for futher preprocessing...",end = "\n\n")

mapping_mi = {
    "Left":0,
    "Right":1
}
inverse_mapping_mi = {
    v:k for k , v in mapping_mi.items()
} 





from utils.preprocessing import SignalPreprocessor
train_mi_new = pd.read_csv("../data_my_head/MI/1/EEGdata.csv")
train_labels_new = pd.read_csv("../data_my_head/MI/1/trial_labels.csv")
val_mi = pd.read_csv("../data_my_head/MI/2/EEGdata.csv")
val_labels = pd.read_csv("../data_my_head/MI/2/trial_labels.csv")


mapping_mi = {
    "Left":0,
    "Right":1
}
trial_df = pd.concat([
    train_mi_new , val_mi
]).reset_index(drop=True)

labels_trial_df = pd.concat([
    train_labels_new , val_labels
]).reset_index(drop=True)








extracting data for futher preprocessing...



In [19]:
def preprocess_fast(
        trial_df,
        labels_df,
        mapping=None,
        task=None,
        signal_processer = None
        ):
    assert task in ["MI" , "SSVEP"]
    if task=="MI":
        eeg_col = ['C3', 'C4', 'CZ', 'FZ' ]
        time_len_per_trial = 2250
    else:
        eeg_col =  ['OZ', 'PO7', 'PO8', 'PZ']
        time_len_per_trial = 1750


    input_array = trial_df.drop(columns = ["Time" , "Battery" , "Counter"])[eeg_col+['AccX',
        'AccY', 'AccZ', 'Gyro1', 'Gyro2', 'Gyro3' , 'Validation']].to_numpy().T
    
    input_array = input_array.reshape(-1 , 11 , time_len_per_trial)
    input_trials = labels_df.direction.map(mapping).values
    subjects = np.full(input_trials.shape, "S13", dtype=object)
    acc_channel = np.linalg.norm(input_array[:,4:7,:],axis = 1)
    gyro_channel = np.linalg.norm(input_array[:,7:10,:],axis = 1)
    Validation_Channel = input_array[:,10,:]
    input_array[:,4,:] = acc_channel
    input_array[:,5,:] = gyro_channel
    input_array[:,6,:] = Validation_Channel

    input_array = input_array[:,:7,:]



    preprocessed_test_data , preprocessed_test_labels , preprocessed_test_subject_ids , weights_test = signal_processer.apply_preprocessing(input_array, input_trials , subjects)
    
    num_windows_per_trial = signal_processer.num_windows_per_trial

    
    return (
        torch.from_numpy(preprocessed_test_data).to(torch.float32),
        torch.from_numpy(preprocessed_test_labels).to(torch.long),
        torch.full(preprocessed_test_subject_ids.shape , 12).to(torch.long),
        torch.from_numpy(weights_test).to(torch.float32),
        num_windows_per_trial
        )

In [20]:
device = torch.device("cuda")

from torchinfo import summary
model_mi_1  = MTCFormer(depth=3,
                    kernel_size=50,
                    modulator_kernel_size=30,
                    n_times=600,
                    chs_num=7,
                    eeg_ch_nums=4,
                    class_num=2,
                    class_num_domain=30,
                    modulator_dropout=0.48929137963218305,
                    mid_dropout=0.5,
                    output_dropout=0.42685917257840517,
                    k=100,
                    projection_dimention=2,
                    seed=4224
                    ).to(device)

for i , param in enumerate(model_mi_1.parameters()):
    if i >80:
        break
    param.requires_grad = False



optimizer = Adam(model_mi_1.parameters(), lr=0.002)

checkpoint_path = os.path.join(
    "..",
    "checkpoints",
    "model_1_mi_checkpoint",
    "best_model_.pth"
    )

checkpoint = torch.load(checkpoint_path, weights_only=False)

model_mi_1.load_state_dict(checkpoint['model_state_dict'] , strict=True)

summary(model_mi_1 , input_size=(1 , 7 , 600))


Layer (type:depth-idx)                             Output Shape              Param #
MTCFormer                                          [1, 2]                    --
├─TemporalModulator: 1-1                           [1, 4, 600]               --
│    └─Conv1d: 2-1                                 [1, 4, 600]               (364)
│    └─ChannelWiseLayerNorm: 2-2                   [1, 4, 600]               (8)
│    └─Dropout: 2-3                                [1, 4, 600]               --
│    └─Sigmoid: 2-4                                [1, 4, 600]               --
├─Conv1d: 1-2                                      [1, 8, 600]               (40)
├─ChannelWiseLayerNorm: 1-3                        [1, 8, 600]               (16)
├─GELU: 1-4                                        [1, 8, 600]               --
├─Dropout: 1-5                                     [1, 8, 600]               --
├─ConvolutionalAttention: 1-6                      [1, 8, 600]               --
│    └─ModuleList: 2-5     

In [21]:


print("Preprocessing data for model 1 ..... ")


preprocessor = SignalPreprocessor(
    fs=250,                                             
    bandpass_low=6.0,                     
    bandpass_high=24.0,                  
    n_cols_to_filter=4,                   
    window_size=600,                      
    window_stride=70,                    
    idx_to_ignore_normalization=-1,        
    crop_range=(2.5 , 7)              
)



input_array , labels , subject_ids , weigths  , num_windows_per_trial = preprocess_fast(
        trial_df,
        labels_trial_df,
        mapping=mapping_mi,
        task="MI",
        signal_processer = preprocessor
        )

Preprocessing data for model 1 ..... 


In [22]:
from utils.CustomDataset import EEGDataset
from utils.augmentation import augment_data


print("Data Preparation.... Wrapping preprocessed data inside tensor datasets....",end = "\n\n")


from torch.utils.data import DataLoader
from utils.CustomDataset import EEGDataset
from torch.nn import *
from torch.optim.lr_scheduler import *
n_trials_to_choose = 12
n_windows_to_choose = num_windows_per_trial*n_trials_to_choose
training_dataset = EEGDataset(
    data_tensor=input_array[:n_windows_to_choose, : , :],
    weigths=weigths[:n_windows_to_choose],
    label_tensor = labels[:n_windows_to_choose],
    subject_labels=subject_ids[:n_windows_to_choose]
    
    )

val_dataset = EEGDataset(
    data_tensor=input_array[n_windows_to_choose:, : , :],
    weigths=weigths[n_windows_to_choose:],
    label_tensor = labels[n_windows_to_choose:],
    subject_labels=subject_ids[n_windows_to_choose:]
)

trial_level_labels = labels[n_windows_to_choose:].clone()

train_loader = DataLoader(training_dataset , batch_size=3 , shuffle = True)
val_loader = DataLoader(val_dataset , batch_size=val_dataset.__len__() , shuffle = False)

Data Preparation.... Wrapping preprocessed data inside tensor datasets....



In [23]:
from torch.nn import CrossEntropyLoss
from torch.optim.lr_scheduler import *
save_path=os.path.join("tuned_checkpoints","model_1_mi")
training_loop_params = {
    "criterion": CrossEntropyLoss(reduction="none"),


    "optimizer_class": Adam,
    "optimizer_config": {
        "lr": 0.002,
        "weight_decay":1.6581884226239174e-05
    },


    "scheduler_class": CosineAnnealingLR,
    "scheduler_config":  {
            "T_max": 121,
            "eta_min": 0.00019632120929158656
        },

    "window_len": num_windows_per_trial,
    "n_epochs": 600,
    "patience": 100 ,
    "domain_lambda": 0.05,
    "lambda_scheduler_fn": None,
    "adversarial_steps": 1,
    "adversarial_epsilon": 0.1,
    "adversarial_alpha": 0.005,
    "adversarial_training": True,
    "save_best_only": True,
    "save_path": save_path,
    "n_classes": 2,
    "device": device,
    "update_loader": (10, 100),
    "scheduler_fn": None ,
    "tensorboard":True,
}

train_model(model_mi_1,
        train_loader=train_loader,
        val_loader=val_loader,
        original_val_labels=trial_level_labels,
        **training_loop_params
    )


Total Parameters of the Model ======> 559,573 paramaters
Launching TensorBoard at http://localhost:6006/ ...
Path Exists. Contents of this folder will be modified save_path is :  tuned_checkpoints/model_1_mi
--- Starting Training Loop ---
✅ Best checkpoint updated (save_best_only=True). at  tuned_checkpoints/model_1_mi
🟢 Validation Balanced Accuracy improved. Saving best model state...
Epoch [1/600] - Train Loss: 138168.0279, Train F1: 0.4353 | adversarial F1 : 0.4103 | Val Loss: 0.6832, Val F1: 0.3826 | Time: 2.3s | Balanced Accuracy Val: 0.5750 | - Domain Loss: 0.2827
✅ Best checkpoint updated (save_best_only=True). at  tuned_checkpoints/model_1_mi
🟢 Validation Balanced Accuracy improved. Saving best model state...
Epoch [2/600] - Train Loss: 120262.3506, Train F1: 0.3615 | adversarial F1 : 0.3407 | Val Loss: 0.6711, Val F1: 0.4333 | Time: 2.5s | Balanced Accuracy Val: 0.6000 | - Domain Loss: 0.0057
🟡 No improvement for 1 epochs.
Epoch [3/600] - Train Loss: 114511.6159, Train F1: 0.3

np.float64(0.5166666666666666)

In [24]:
print("extracting data for futher preprocessing...",end = "\n\n")

mapping_ssvep = {
    "Backward":0,
    "Forward":1,
    "Left":2,
    "Right":3
}
inverse_mapping_mi = {
    v:k for k , v in mapping_mi.items()
} 





from utils.preprocessing import SignalPreprocessor
train_mi_new = pd.read_csv("../data_my_head/SSVEP/1/EEGdata.csv")
train_labels_new = pd.read_csv("../data_my_head/SSVEP/1/trial_labels.csv")
val_mi = pd.read_csv("../data_my_head/SSVEP/2/EEGdata.csv")
val_labels = pd.read_csv("../data_my_head/SSVEP/2/trial_labels.csv")


mapping_ssvep = {
    "Backward":0,
    "Forward":1,
    "Left":2,
    "Right":3
}
trial_df = pd.concat([
    train_mi_new , val_mi
]).reset_index(drop=True)

labels_trial_df = pd.concat([
    train_labels_new , val_labels
]).reset_index(drop=True)


preprocessor = SignalPreprocessor(
    fs=250,                                                 
    bandpass_low=8,                     
    bandpass_high=14,                  
    n_cols_to_filter=4,                   
    window_size=500,                      
    window_stride=50,                    
    idx_to_ignore_normalization=-1,        
    crop_range=(1.5 , 6)            
)


input_array , labels , subject_ids , weigths  , num_windows_per_trial = preprocess_fast(
        trial_df,
        labels_trial_df,
        mapping=mapping_ssvep,
        task="SSVEP",
        signal_processer = preprocessor
        )


extracting data for futher preprocessing...



array([2, 0, 3, 3, 3, 2, 3, 3, 2, 1])

In [109]:
from utils.CustomDataset import EEGDataset
from utils.augmentation import augment_data


print("Data Preparation.... Wrapping preprocessed data inside tensor datasets....",end = "\n\n")


from torch.utils.data import DataLoader
from utils.CustomDataset import EEGDataset
from torch.nn import *
from torch.optim.lr_scheduler import *
n_trials_to_choose = 12
n_windows_to_choose = num_windows_per_trial*n_trials_to_choose
training_dataset = EEGDataset(
    data_tensor=input_array[:n_windows_to_choose, : , :],
    weigths=weigths[:n_windows_to_choose],
    label_tensor = labels[:n_windows_to_choose],
    subject_labels=subject_ids[:n_windows_to_choose]
    
    )

val_dataset = EEGDataset(
    data_tensor=input_array[n_windows_to_choose:, : , :],
    weigths=weigths[n_windows_to_choose:],
    label_tensor = labels[n_windows_to_choose:],
    subject_labels=subject_ids[n_windows_to_choose:]
)

trial_level_labels = val_labels.direction.map(mapping_ssvep).values

train_loader = DataLoader(training_dataset , batch_size=3 , shuffle = True)
val_loader = DataLoader(val_dataset , batch_size=val_dataset.__len__() , shuffle = False)

Data Preparation.... Wrapping preprocessed data inside tensor datasets....



In [59]:
labels[:n_windows_to_choose:]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [63]:
from model.MTCformerV2 import MTCFormer
from torch.optim.lr_scheduler import *
from utils.training import train_model 
model_ssvep = MTCFormer(
    depth=5,
    kernel_size=50,
    n_times=500,
    chs_num=7,
    eeg_ch_nums=4,
    class_num=4,
    class_num_domain=30,
    modulator_kernel_size=30,
    domain_dropout=0.4,
    modulator_dropout=0.4,
    mid_dropout=0.4,
    output_dropout=0.6,
    k=100,
    projection_dimention=2,
    seed = 5445
).to(device)
optimizer = Adam(model_ssvep.parameters(), lr=0.002)

checkpoint_path = os.path.join(
    "..",
    "checkpoints",
    "model_ssvep_checkpoint",
    "best_model_.pth"
    )

checkpoint = torch.load(checkpoint_path, weights_only=False)

model_ssvep.load_state_dict(checkpoint['model_state_dict'] , strict=False)


for i , param in enumerate(model_mi_1.parameters()):
    if i >50:
        break
    param.requires_grad = False

In [64]:
save_path = os.path.join("","tuned_checkpoints","model_ssvep")

training_loop_params = {
    "criterion": CrossEntropyLoss(reduction="none"),


    "optimizer_class": Adam,
    "optimizer_config": {
        "lr": 0.001,
        "weight_decay":0
    },


    "scheduler_class": None,
    "scheduler_config":  {
            "T_max": 121,
            "eta_min": 0.00019632120929158656
        },

    "window_len": num_windows_per_trial,
    "n_epochs": 600,
    "patience": 600 ,
    "domain_lambda": 0.2,
    "lambda_scheduler_fn": None,
    "adversarial_steps": 1,
    "adversarial_epsilon": 0.05,
    "adversarial_alpha": 0.005,
    "adversarial_training": True,
    "save_best_only": True,
    "save_path": save_path,
    "n_classes": 4,
    "device": device,
    "update_loader": (20, 100),
    "scheduler_fn": None  # Optional: your own dynamic LR adjustment function
}

train_model(model_ssvep,
            train_loader=train_loader,
            val_loader=val_loader,
            original_val_labels=trial_level_labels,
            **training_loop_params
    )

Total Parameters of the Model ======> 1,351,110 paramaters
Path Exists. Contents of this folder will be modified save_path is :  tuned_checkpoints/model_ssvep
--- Starting Training Loop ---


/home/mohammed_ahmed/miniconda3/envs/Competition_environment/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


✅ Best checkpoint updated (save_best_only=True). at  tuned_checkpoints/model_ssvep
🟢 Validation Balanced Accuracy improved. Saving best model state...
Epoch [1/600] - Train Loss: 202359.8548, Train F1: 0.3177 | adversarial F1 : 0.3061 | Val Loss: 1.4450, Val F1: 0.0000 | Time: 2.3s | Balanced Accuracy Val: 0.0000 | - Domain Loss: 0.1665


/home/mohammed_ahmed/miniconda3/envs/Competition_environment/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


✅ Best checkpoint updated (save_best_only=True). at  tuned_checkpoints/model_ssvep
🟢 Validation Balanced Accuracy improved. Saving best model state...
Epoch [2/600] - Train Loss: 128734.8483, Train F1: 0.3544 | adversarial F1 : 0.3563 | Val Loss: 1.4602, Val F1: 0.0373 | Time: 1.9s | Balanced Accuracy Val: 0.0103 | - Domain Loss: 0.0291


...

🟡 No improvement for 94 epochs.
Epoch [97/600] - Train Loss: 1059.3993, Train F1: 0.5504 | adversarial F1 : 0.5466 | Val Loss: 1.6469, Val F1: 0.0373 | Time: 0.1s | Balanced Accuracy Val: 0.0103 | - Domain Loss: 0.0004
🟡 No improvement for 95 epochs.
Epoch [98/600] - Train Loss: 908.9614, Train F1: 0.5288 | adversarial F1 : 0.5339 | Val Loss: 1.6463, Val F1: 0.0373 | Time: 0.1s | Balanced Accuracy Val: 0.0103 | - Domain Loss: 0.0002


/home/mohammed_ahmed/miniconda3/envs/Competition_environment/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/mohammed_ahmed/miniconda3/envs/Competition_environment/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


🟡 No improvement for 96 epochs.
Epoch [99/600] - Train Loss: 1138.6711, Train F1: 0.5237 | adversarial F1 : 0.5258 | Val Loss: 1.6458, Val F1: 0.0373 | Time: 0.1s | Balanced Accuracy Val: 0.0103 | - Domain Loss: 0.0012
🟡 No improvement for 97 epochs.
Epoch [100/600] - Train Loss: 1457.7355, Train F1: 0.5423 | adversarial F1 : 0.5322 | Val Loss: 1.6454, Val F1: 0.0373 | Time: 0.1s | Balanced Accuracy Val: 0.0103 | - Domain Loss: 0.0005
🟡 No improvement for 98 epochs.
Epoch [101/600] - Train Loss: 1341.6254, Train F1: 0.5420 | adversarial F1 : 0.5266 | Val Loss: 1.6451, Val F1: 0.0373 | Time: 0.1s | Balanced Accuracy Val: 0.0103 | - Domain Loss: 0.0002
---BREAKING TRAINING----


/home/mohammed_ahmed/miniconda3/envs/Competition_environment/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


np.float64(0.04648829431438127)

In [101]:
np.set_printoptions(suppress=True)



In [103]:
trial_level_labels

tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])

In [102]:
predict_optimized(
    model_ssvep,
    windows_per_trial=num_windows_per_trial,
    loader=val_loader,
    probability=True
)

array([[0.32958537, 0.03263313, 0.38126412, 0.25651738],
       [0.3255843 , 0.18979754, 0.31974265, 0.16487542],
       [0.4849823 , 0.00704174, 0.48517603, 0.02280002],
       [0.5536281 , 0.04972099, 0.36318922, 0.03346165],
       [0.03661281, 0.8781843 , 0.03940997, 0.04579278],
       [0.01608884, 0.95102745, 0.0132519 , 0.01963184],
       [0.00012568, 0.99934286, 0.0001578 , 0.0003736 ],
       [0.00014678, 0.00019971, 0.0002484 , 0.99940515]], dtype=float32)

In [83]:
labels_trial_df

,trial,direction
0,1,Forward
1,2,Right
2,3,Forward
3,4,Left
4,5,Right
5,6,Left
6,7,Forward
7,8,Backward
8,9,Right
9,10,Backward


In [92]:
n = 8
predict(model_ssvep.to(torch.device("cpu")) , trial_df[n*1750:(n+1)*1750])

'Right'

In [78]:
def predict(model, df):
    """
    Preprocess the raw MI data and make a prediction.

    Args:
        model: The loaded model object.
        df (pd.DataFrame): The raw trial data.

    Returns:
        str: The prediction, which must be "left", "right", or "?".
    """
    def preprocess_optimized(
            trial_df,
            signal_processer = None
            ):
        eeg_col =  ['OZ', 'PO7', 'PO8', 'PZ']


        input_array = trial_df.drop(columns = ["Time" , "Battery" , "Counter"])[eeg_col+['AccX',
            'AccY', 'AccZ', 'Gyro1', 'Gyro2', 'Gyro3' , 'Validation']].to_numpy().T
        
        acc_channel = np.linalg.norm(input_array[4:7,:],axis = 0)
        gyro_channel = np.linalg.norm(input_array[7:10,:],axis = 0)
        Validation_Channel = input_array[10,:]
        input_array[4,:] = acc_channel
        input_array[5,:] = gyro_channel
        input_array[6,:] = Validation_Channel

        input_array = input_array[:7,:]


        preprocessed_test_data , _ , _ , weights_test = signal_processer.apply_preprocessing(np.expand_dims(input_array,axis=0), np.array([None]) , np.array([1]))
        
        num_windows_per_trial = signal_processer.num_windows_per_trial

        
        return (
            torch.from_numpy(preprocessed_test_data).to(torch.float32),
           -1,
            -1,
            torch.from_numpy(weights_test).to(torch.float32),
            num_windows_per_trial
            )

    preprocessor = SignalPreprocessor(
    fs=250,                                                 
    bandpass_low=8,                     
    bandpass_high=14,                  
    n_cols_to_filter=4,                   
    window_size=500,                      
    window_stride=50,                    
    idx_to_ignore_normalization=-1,        
    crop_range=(1.5 , 6)            
)
    data , _ , _ , weights , windows_per_trial = preprocess_optimized(df,signal_processer=preprocessor)

    
    index_to_label = {
        0: "Backward",
        1: "Forward",
        2: "Left",
        3: "Right"
    }

    prediction = predict_optimized(
        model=model,
        windows_per_trial=windows_per_trial,
        loader= (data , weights)

    )

    return index_to_label[prediction[0]]